<a href="https://colab.research.google.com/github/pamelag/level-up-ai/blob/main/LLM_Expr7_101(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install torch
!pip install transformers
!pip install accelerate
!pip install bitsandbytes

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoTokenizer

MODEL_NAME = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
text = "The most important person in AI is"
encoding = tokenizer(text, return_tensors="pt")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, GenerationConfig

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, device_map="auto", torch_dtype=torch.float16
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 128
generation_config.repetition_penalty = 1.18
generation_config.temperature = 0.0000001

In [ ]:
import torch
from transformers import AutoModelForCausalLM, GenerationConfig

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, device_map="auto", torch_dtype=torch.float16
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 128
generation_config.repetition_penalty = 1.18
generation_config.temperature = 0.0000001

In [ ]:
!nvidia-smi

In [ ]:
encoding = encoding.to(model.device)

In [ ]:
%%time
with torch.no_grad():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )

In [ ]:
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [ ]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, device_map="auto", torch_dtype=torch.float16
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 512
generation_config.repetition_penalty = 1.18
generation_config.temperature = 0.0000001

In [ ]:
from transformers import TextStreamer, pipeline

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    num_return_sequences=1,
    streamer=streamer,
)

In [ ]:
output = llm("Who is the most important person in AI?")

In [ ]:
system_prompt = """
You are a professional writer. Rewrite the following text in simple words.
"""

In [ ]:
text = '''Strategy & Innovation team is working on innovative intelligent document processing capabilities as part of Implement Intelligent Automation Solutions Aspire Initiative under Strong,
Secure Digital Foundation strategic Program and Embrace Innovation Focus Area. The initiative lead of implement intelligent automation solutions is Jocelyn Yem, Director, Strategy & Innovation.
High Level Action for this initiative is to Implement innovative intelligent document processing capabilities. Major Milestones - Completed Proof of Value work and select Proof of Concept tool(s),
Expanded intelligent document processing capability across Revenue NSW (inc. Luminance). Developed scaled Proof of Concept'''

prompt = f"""
Analyze the text and write the main points.
```
{text}
```
""".strip()

In [ ]:
system_prompt = """
You are a professional writer. Rewrite the following text in formal words.
"""

In [ ]:
text = '''Employee Voice is an Aspire Initiative under Strong, Future Fit Workforce strategic Program and People at the Heart Strategic Focus Area. The initiative lead of strategic workforce plan solutions is
    Scott Johnston, Deputy Secretary, Revenue NSW. The High Level actions are to Derive insights from the organisational People Matters Engagement Survey (PMES) results & implement an action plan to focus on areas for opportunity
    Major Milestones - Delivered focused PMES communication plan, Developed PMES action plans in each business unit and across Revenue NSW,
    Implemented Revenue NSW PMES action plans, using existing feedback mechanisms where possible to evaluate action.'''

prompt = f"""
Analyze the text and write the main points.
```
{text}
```
""".strip()

In [ ]:
from typing import Optional


def predict(prompt: str, system_prompt: Optional[str] = None):
    messages = [
        {
            "role": "user",
            "content": prompt,
        }
    ]
    if system_prompt:
        messages.insert(0, {"role": "system", "content": system_prompt})
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    return llm(prompt)

In [ ]:
%%time
output = predict(prompt, system_prompt)

In [ ]:
model.save_pretrained("*/models/meta-llama/Llama-2-7b-chat-hf")

# Faster Inference

In [ ]:
# MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME, device_map="auto", torch_dtype=torch.float16
# )

# generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
# generation_config.max_new_tokens = 512
# generation_config.repetition_penalty = 1.18
# generation_config.temperature = 0.0000001

In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True,
    load_in_8bit=True,
)